## 1️⃣ Setup & Data Loading

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path

# Time series libraries
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller

# Metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

# Settings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Libraries loaded successfully!")

In [ ]:
# Load time series data
df_state = pd.read_csv('cleaned_data/state_timeseries.csv')
df_national = pd.read_csv('cleaned_data/national_timeseries.csv')

print("📊 Data loaded successfully!")
print(f"\nState-level: {len(df_state):,} rows")
print(f"States: {df_state['state'].nunique()}")
print(f"Years: {df_state['year'].nunique()}")

print(f"\nNational-level: {len(df_national):,} rows")
print(f"Years: {df_national['year'].nunique()}")

In [ ]:
# Preview data
print("State-level data:")
display(df_state.head(10))

print("\nNational-level data:")
display(df_national)

---
## 2️⃣ Exploratory Time Series Analysis

In [ ]:
# National trend visualization
plt.figure(figsize=(14, 6))
plt.plot(df_national['year'], df_national['crimes'], marker='o', linewidth=2, markersize=8, color='#e74c3c')
plt.title('National Property Crime Trend (2016-2023)', fontsize=16, fontweight='bold')
plt.xlabel('Year', fontsize=12)
plt.ylabel('Total Property Crimes', fontsize=12)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("\n📈 Trend Analysis:")
print(f"Starting value (2016): {df_national.iloc[0]['crimes']:,.0f}")
print(f"Ending value (2023): {df_national.iloc[-1]['crimes']:,.0f}")
print(f"Overall change: {((df_national.iloc[-1]['crimes'] / df_national.iloc[0]['crimes']) - 1) * 100:.1f}%")

In [ ]:
# Top 5 states by total crime
state_totals = df_state.groupby('state')['crimes'].sum().sort_values(ascending=False).head(5)

print("🏆 Top 5 States by Total Property Crimes (2016-2023):")
for i, (state, total) in enumerate(state_totals.items(), 1):
    print(f"{i}. {state}: {total:,.0f}")

In [ ]:
# Visualize top 5 states trends
top_states = state_totals.index.tolist()
df_top_states = df_state[df_state['state'].isin(top_states)]

plt.figure(figsize=(14, 8))
for state in top_states:
    state_data = df_top_states[df_top_states['state'] == state]
    plt.plot(state_data['year'], state_data['crimes'], marker='o', label=state, linewidth=2)

plt.title('Property Crime Trends: Top 5 States (2016-2023)', fontsize=16, fontweight='bold')
plt.xlabel('Year', fontsize=12)
plt.ylabel('Property Crimes', fontsize=12)
plt.legend(loc='best', fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

---
## 3️⃣ Stationarity Testing

**Purpose:** Check if time series is stationary (constant mean, variance, autocorrelation)

**Method:** Augmented Dickey-Fuller (ADF) test

**Interpretation:** p-value < 0.05 = stationary (good for ARIMA)

In [ ]:
# Test stationarity for national series
def test_stationarity(series, series_name):
    """
    Perform Augmented Dickey-Fuller test for stationarity
    """
    result = adfuller(series.dropna())
    
    print(f"\n{'='*60}")
    print(f"Stationarity Test: {series_name}")
    print(f"{'='*60}")
    print(f"ADF Statistic: {result[0]:.4f}")
    print(f"p-value: {result[1]:.4f}")
    print(f"Critical Values:")
    for key, value in result[4].items():
        print(f"  {key}: {value:.4f}")
    
    if result[1] < 0.05:
        print(f"\n✅ Series is STATIONARY (p < 0.05)")
        print("   → No differencing needed for ARIMA")
    else:
        print(f"\n⚠️  Series is NON-STATIONARY (p >= 0.05)")
        print("   → Differencing recommended (d=1 in ARIMA)")
    
    return result[1] < 0.05

# Test national series
is_stationary_national = test_stationarity(df_national['crimes'], "National Property Crimes")

In [ ]:
# Test stationarity for top 3 states
top_3_states = state_totals.head(3).index.tolist()

stationarity_results = {}
for state in top_3_states:
    state_series = df_state[df_state['state'] == state]['crimes']
    is_stationary = test_stationarity(state_series, f"{state} State")
    stationarity_results[state] = is_stationary

---
## 4️⃣ ACF/PACF Analysis

**Purpose:** Determine ARIMA parameters (p, d, q)

- **ACF (Autocorrelation Function):** Helps determine q (MA order)
- **PACF (Partial Autocorrelation Function):** Helps determine p (AR order)
- **d (Differencing):** Determined by stationarity test

In [ ]:
# Plot ACF and PACF for national series
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

plot_acf(df_national['crimes'], lags=6, ax=axes[0])
axes[0].set_title('Autocorrelation Function (ACF)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Lag')

plot_pacf(df_national['crimes'], lags=6, ax=axes[1])
axes[1].set_title('Partial Autocorrelation Function (PACF)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Lag')

plt.tight_layout()
plt.show()

print("\n📊 Interpretation Guide:")
print("  • ACF: If cuts off after lag q → MA(q) component")
print("  • PACF: If cuts off after lag p → AR(p) component")
print("  • With only 8 data points, keep p and q small (0-2)")

---
## 5️⃣ ARIMA Modeling - National Level

**Challenge:** Only 8 years of data = limited forecasting capability

**Strategy:**
- Use simple ARIMA models (low p, d, q)
- Train on 2016-2022 (7 years)
- Test on 2023 (1 year)
- Forecast 2024-2025 (2 years ahead)

In [ ]:
# Prepare train-test split
train_national = df_national[df_national['year'] < 2023].copy()
test_national = df_national[df_national['year'] == 2023].copy()

print(f"Training data: {len(train_national)} years (2016-2022)")
print(f"Test data: {len(test_national)} year (2023)")
print(f"\nWill forecast 2 additional years: 2024-2025")

In [ ]:
# Try multiple ARIMA configurations
arima_configs = [
    (0, 0, 0),  # Baseline (mean)
    (1, 0, 0),  # AR(1)
    (0, 1, 0),  # I(1) - first difference
    (1, 1, 0),  # ARIMA(1,1,0)
    (0, 1, 1),  # ARIMA(0,1,1)
    (1, 1, 1),  # ARIMA(1,1,1)
]

results_national = []

print("🔮 Testing ARIMA configurations...\n")
print(f"{'Config':<15} {'MAE':<12} {'RMSE':<12} {'MAPE':<12} {'Status'}")
print("="*65)

for order in arima_configs:
    try:
        # Fit model
        model = ARIMA(train_national['crimes'], order=order)
        fitted = model.fit()
        
        # Forecast 1 step (2023)
        forecast = fitted.forecast(steps=1)[0]
        
        # Calculate metrics
        actual = test_national['crimes'].values[0]
        mae = abs(forecast - actual)
        rmse = np.sqrt((forecast - actual)**2)
        mape = abs((actual - forecast) / actual) * 100
        
        results_national.append({
            'order': order,
            'mae': mae,
            'rmse': rmse,
            'mape': mape,
            'forecast_2023': forecast,
            'model': fitted
        })
        
        print(f"ARIMA{order:<8} {mae:<12.0f} {rmse:<12.0f} {mape:<12.1f}% ✓")
        
    except Exception as e:
        print(f"ARIMA{order:<8} {'Failed':<12} {'Failed':<12} {'Failed':<12} ✗")

# Find best model
best_model = min(results_national, key=lambda x: x['mape'])
print("\n" + "="*65)
print(f"✨ Best Model: ARIMA{best_model['order']} (MAPE: {best_model['mape']:.1f}%)")

In [ ]:
# Forecast 2024-2025 using best model
best_fitted = best_model['model']

# Refit on full dataset (2016-2023) for final forecast
final_model = ARIMA(df_national['crimes'], order=best_model['order'])
final_fitted = final_model.fit()

# Forecast 2 years ahead
forecast_2024_2025 = final_fitted.forecast(steps=2)
forecast_ci = final_fitted.get_forecast(steps=2).conf_int(alpha=0.05)

print("🔮 Property Crime Forecast (National Level)")
print("="*60)
print(f"2024: {forecast_2024_2025.iloc[0]:,.0f} crimes")
print(f"      95% CI: [{forecast_ci.iloc[0, 0]:,.0f}, {forecast_ci.iloc[0, 1]:,.0f}]")
print(f"\n2025: {forecast_2024_2025.iloc[1]:,.0f} crimes")
print(f"      95% CI: [{forecast_ci.iloc[1, 0]:,.0f}, {forecast_ci.iloc[1, 1]:,.0f}]")

# Calculate change
change_2024 = ((forecast_2024_2025.iloc[0] / df_national.iloc[-1]['crimes']) - 1) * 100
change_2025 = ((forecast_2024_2025.iloc[1] / forecast_2024_2025.iloc[0]) - 1) * 100

print(f"\n📊 Expected Change:")
print(f"2023 → 2024: {change_2024:+.1f}%")
print(f"2024 → 2025: {change_2025:+.1f}%")

In [ ]:
# Visualize forecast
plt.figure(figsize=(14, 7))

# Historical data
plt.plot(df_national['year'], df_national['crimes'], 
         marker='o', linewidth=2, markersize=8, label='Historical (2016-2023)', color='#3498db')

# Forecast
forecast_years = [2024, 2025]
plt.plot(forecast_years, forecast_2024_2025, 
         marker='s', linewidth=2, markersize=8, label='Forecast (2024-2025)', 
         color='#e74c3c', linestyle='--')

# Confidence interval
plt.fill_between(forecast_years, 
                 forecast_ci.iloc[:, 0], 
                 forecast_ci.iloc[:, 1], 
                 alpha=0.2, color='#e74c3c', label='95% Confidence Interval')

plt.title(f'National Property Crime Forecast\nARIMA{best_model["order"]} Model', 
          fontsize=16, fontweight='bold')
plt.xlabel('Year', fontsize=12)
plt.ylabel('Total Property Crimes', fontsize=12)
plt.legend(loc='best', fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

---
## 6️⃣ State-Level Forecasting

**Focus:** Top 3 states by total crime volume

In [ ]:
# Forecast top 3 states
top_3_states = state_totals.head(3).index.tolist()
state_forecasts = {}

print("🔮 State-Level Forecasts (2024-2025)\n")
print(f"{'State':<20} {'2023':<12} {'2024':<12} {'2025':<12} {'Change 24-25'}")
print("="*75)

for state in top_3_states:
    # Get state data
    state_data = df_state[df_state['state'] == state].sort_values('year')
    
    # Use best ARIMA order from national model
    try:
        model = ARIMA(state_data['crimes'], order=best_model['order'])
        fitted = model.fit()
        forecast = fitted.forecast(steps=2)
        
        state_forecasts[state] = {
            'historical': state_data,
            'forecast_2024': forecast.iloc[0],
            'forecast_2025': forecast.iloc[1],
            'actual_2023': state_data.iloc[-1]['crimes']
        }
        
        change = ((forecast.iloc[1] / forecast.iloc[0]) - 1) * 100
        
        print(f"{state:<20} {state_data.iloc[-1]['crimes']:<12.0f} {forecast.iloc[0]:<12.0f} "
              f"{forecast.iloc[1]:<12.0f} {change:+.1f}%")
        
    except Exception as e:
        print(f"{state:<20} {'Failed to forecast':<50}")

In [ ]:
# Visualize state forecasts
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, state in enumerate(top_3_states):
    if state in state_forecasts:
        data = state_forecasts[state]
        
        # Historical
        axes[idx].plot(data['historical']['year'], data['historical']['crimes'], 
                      marker='o', linewidth=2, label='Historical', color='#3498db')
        
        # Forecast
        forecast_years = [2024, 2025]
        forecast_values = [data['forecast_2024'], data['forecast_2025']]
        axes[idx].plot(forecast_years, forecast_values, 
                      marker='s', linewidth=2, label='Forecast', 
                      color='#e74c3c', linestyle='--')
        
        axes[idx].set_title(f'{state}', fontsize=12, fontweight='bold')
        axes[idx].set_xlabel('Year')
        axes[idx].set_ylabel('Property Crimes')
        axes[idx].legend()
        axes[idx].grid(alpha=0.3)

plt.suptitle('State-Level Property Crime Forecasts (Top 3 States)', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

---
## 7️⃣ Model Evaluation Summary

In [ ]:
print("""\n
╔════════════════════════════════════════════════════════════════════╗
║              ✅ TIME SERIES FORECASTING COMPLETED                  ║
╚════════════════════════════════════════════════════════════════════╝

📊 MODELS TRAINED:

1. National Level
   • Best Model: ARIMA{}
   • Test MAPE: {:.1f}%
   • 2024 Forecast: {:,.0f} crimes
   • 2025 Forecast: {:,.0f} crimes

2. State Level (Top 3)
   • {} states forecasted
   • All using ARIMA{}
   • 2-year horizon (2024-2025)

⚠️  LIMITATIONS:
• Only 8 years of historical data (2016-2023)
• Simple ARIMA models due to limited observations
• No seasonal decomposition (yearly data only)
• Forecasts should be interpreted with caution

💡 RECOMMENDATIONS:
• Use forecasts for general trend indication, not precise predictions
• Consider ensemble with other methods (exponential smoothing)
• Update models annually as more data becomes available
• Focus on trend direction rather than exact values

🎯 NEXT STEPS:
✓ Use forecasts in PowerBI dashboard (trend indicators)
✓ Compare with clustering results (high-crime districts)
✓ Integrate with classification model (risk predictions)
""" .format(
    best_model['order'],
    best_model['mape'],
    forecast_2024_2025.iloc[0],
    forecast_2024_2025.iloc[1],
    len(state_forecasts),
    best_model['order']
))

---
## 8️⃣ Export Forecast Results

In [ ]:
# Create forecast output directory
output_dir = Path('forecasts')
output_dir.mkdir(exist_ok=True)

# Export national forecast
national_forecast_df = pd.DataFrame({
    'year': [2024, 2025],
    'forecast': forecast_2024_2025.values,
    'lower_ci': forecast_ci.iloc[:, 0].values,
    'upper_ci': forecast_ci.iloc[:, 1].values,
    'level': 'National'
})

national_forecast_df.to_csv(output_dir / 'national_forecast_2024_2025.csv', index=False)
print(f"✓ Saved: national_forecast_2024_2025.csv")

# Export state forecasts
state_forecast_rows = []
for state, data in state_forecasts.items():
    state_forecast_rows.append({
        'state': state,
        'year': 2024,
        'forecast': data['forecast_2024']
    })
    state_forecast_rows.append({
        'state': state,
        'year': 2025,
        'forecast': data['forecast_2025']
    })

state_forecast_df = pd.DataFrame(state_forecast_rows)
state_forecast_df.to_csv(output_dir / 'state_forecasts_2024_2025.csv', index=False)
print(f"✓ Saved: state_forecasts_2024_2025.csv")

print(f"\n📁 All forecasts saved to: {output_dir.absolute()}")

---
## ✅ Summary

**Completed:**
- ✓ Loaded and analyzed property crime time series (2016-2023)
- ✓ Tested stationarity and visualized ACF/PACF patterns
- ✓ Trained multiple ARIMA configurations
- ✓ Selected best model based on test performance
- ✓ Forecasted national property crimes for 2024-2025
- ✓ Forecasted top 3 states individually
- ✓ Generated visualizations with confidence intervals
- ✓ Exported forecast results for PowerBI integration

**Key Insights:**
- National property crime trend shows [direction based on forecast]
- Model performance acceptable given limited data (8 years)
- State-level patterns vary significantly
- Forecasts provide directional guidance for resource allocation

**Limitations:**
- Small sample size limits model complexity
- No seasonal patterns (yearly granularity only)
- Forecast uncertainty increases with horizon

**Next:** Clustering analysis to identify crime hotspots